# 1) Data Preperation

In [ ]:
%pip install ffmpeg

In [ ]:
import os
import librosa
import soundfile as sf
from tqdm import tqdm
from moviepy.video.io.VideoFileClip import VideoFileClip

# Define paths
DATA_DIRS = {
    "dev": "C:/Users/hp333/Desktop/Multimodel_emotion_detection/data/MELD.Raw/dev/dev_splits_complete",
    "test": "C:/Users/hp333/Desktop/Multimodel_emotion_detection/data/MELD.Raw/test/output_repeated_splits_test",
    "train": "C:/Users/hp333/Desktop/Multimodel_emotion_detection/data/MELD.Raw/train/train_splits"
}
OUTPUT_BASE = "C:/Users/hp333/Desktop/Multimodel_emotion_detection/Model/Notebooks/meld_audio"

os.makedirs(OUTPUT_BASE, exist_ok=True)

bad_files = []

import subprocess

def extract_audio(video_path, out_path):
    try:
        cmd = [
            "ffmpeg", "-y",
            "-i", video_path,
            "-vn",
            "-ac", "1",
            "-ar", "16000",
            "-acodec", "pcm_s16le",
            out_path
        ]
        subprocess.run(
            cmd,
            check=True,
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL
        )
        return True
    except Exception as e:
        bad_files.append((video_path, str(e)))
        return False


for split, in_dir in DATA_DIRS.items():
    out_dir = os.path.join(OUTPUT_BASE, split)
    os.makedirs(out_dir, exist_ok=True)
    
    video_files = [
    f for f in os.listdir(in_dir)
    if f.endswith(".mp4") and not f.startswith("._")]
    print(f"\n🎬 Processing {len(video_files)} files in {split} split...")
    
    success, fail = 0, 0
    for f in tqdm(video_files):
        in_path = os.path.join(in_dir, f)
        out_path = os.path.join(out_dir, f.replace(".mp4", ".wav"))
        if extract_audio(in_path, out_path):
            success += 1
        else:
            fail += 1

    print(f"✅ {success} extracted | ❌ {fail} failed in {split} split.")

print("\n🚨 Bad files logged:")
for f, err in bad_files[:10]:
    print(f"- {f}: {err}")
print(f"\nTotal bad files: {len(bad_files)}")

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, Wav2Vec2FeatureExtractor, Wav2Vec2Model
import pandas as pd
import numpy as np
import librosa
import os
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
import torch.nn.functional as F

c:\Users\hp333\Desktop\Multimodel_emotion_detection\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MAX_AUDIO_LEN = 16000 * 5  # 5 sec clips
MAX_TEXT_LEN = 64
BATCH_SIZE = 8
EPOCHS = 15
LR = 2e-5

In [3]:
text_model_name = "distilbert-base-uncased"
audio_model_name = "facebook/wav2vec2-base-960h"

text_tokenizer = AutoTokenizer.from_pretrained(text_model_name)
text_encoder = AutoModel.from_pretrained(text_model_name).to(DEVICE)

audio_extractor = Wav2Vec2FeatureExtractor.from_pretrained(audio_model_name)
audio_encoder = Wav2Vec2Model.from_pretrained(audio_model_name).to(DEVICE)

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
def pad_or_truncate_np(audio_array, target_len):
    """Ensure audio array is exactly target_len long."""
    if len(audio_array) > target_len:
        return audio_array[:target_len]
    elif len(audio_array) < target_len:
        return np.pad(audio_array, (0, target_len - len(audio_array)))
    return audio_array

In [5]:
class MELDFusionDataset(Dataset):
    def __init__(self, csv_path, audio_dir, tokenizer, feature_extractor, label_encoder, max_text_len=64):
        self.df = pd.read_csv(csv_path)
        self.audio_dir = audio_dir
        self.tokenizer = tokenizer
        self.feature_extractor = feature_extractor
        self.label_encoder = label_encoder
        self.max_text_len = max_text_len
        self.df = self.df[self.df['Utterance'].notnull()]  # clean missing text

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        utt = row["Utterance"]
        label = self.label_encoder.transform([row["Emotion"]])[0]
        dia, utt_id = row["Dialogue_ID"], row["Utterance_ID"]
        audio_path = os.path.join(self.audio_dir, f"dia{dia}_utt{utt_id}.wav")

        # --- TEXT ---
        text_inputs = self.tokenizer(
            utt,
            truncation=True,
            padding="max_length",
            max_length=self.max_text_len,
            return_tensors="pt"
        )

        # --- AUDIO ---
        if os.path.exists(audio_path):
            try:
                audio, _ = librosa.load(audio_path, sr=16000)
                audio = pad_or_truncate_np(audio, MAX_AUDIO_LEN)
            except Exception:
                audio = np.zeros(MAX_AUDIO_LEN)
        else:
            audio = np.zeros(MAX_AUDIO_LEN)

        # ✅ Explicitly wrap in list to force feature extractor padding uniformity
        audio_inputs = self.feature_extractor(
            [audio], sampling_rate=16000, return_tensors="pt", padding=True
        )

        return {
            "input_ids": text_inputs["input_ids"].squeeze(0),
            "attention_mask": text_inputs["attention_mask"].squeeze(0),
            "audio_values": audio_inputs["input_values"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long)
        }

In [6]:
class FusionClassifier(nn.Module):
    def __init__(self, text_dim=768, audio_dim=768, hidden_dim=512, num_classes=7):
        super().__init__()
        self.text_proj = nn.Linear(text_dim, hidden_dim)
        self.audio_proj = nn.Linear(audio_dim, hidden_dim)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_dim * 2, num_classes)

    def forward(self, text_emb, audio_emb):
        t = torch.relu(self.text_proj(text_emb))
        a = torch.relu(self.audio_proj(audio_emb))
        fused = torch.cat([t, a], dim=1)
        fused = self.dropout(fused)
        return self.fc(fused)

In [7]:
train_csv = "C:/Users/hp333/Desktop/Multimodel_emotion_detection/data/MELD.Raw/train/train_sent_emo.csv"
dev_csv = "C:/Users/hp333/Desktop/Multimodel_emotion_detection/data/MELD.Raw/dev/dev_sent_emo.csv"

le = LabelEncoder()
train_df = pd.read_csv(train_csv)
le.fit(train_df["Emotion"])

train_set = MELDFusionDataset(train_csv, "C:/Users/hp333/Desktop/Multimodel_emotion_detection/data/meld_audio/train", text_tokenizer, audio_extractor, le)
dev_set = MELDFusionDataset(dev_csv, "C:/Users/hp333/Desktop/Multimodel_emotion_detection/data/meld_audio/dev", text_tokenizer, audio_extractor, le)

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
dev_loader = DataLoader(dev_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [8]:
def compute_text_embeddings(input_ids, attention_mask):
    with torch.no_grad():
        outputs = text_encoder(input_ids.to(DEVICE), attention_mask=attention_mask.to(DEVICE))
        return outputs.last_hidden_state[:, 0, :]  # CLS token


def compute_audio_embeddings(audio_values):
    with torch.no_grad():
        outputs = audio_encoder(audio_values.to(DEVICE))
        return outputs.last_hidden_state.mean(dim=1)

In [9]:
def evaluate(model, data_loader, epoch):
    criterion = nn.CrossEntropyLoss()
    model.eval()
    y_true, y_pred = [], []
    total_loss = 0.0

    with torch.no_grad():
        for batch in tqdm(data_loader,desc=f"Epoch {epoch+1}/{EPOCHS} [Dev]"):
            # move batch to device
            batch = {k: v.to(DEVICE) if torch.is_tensor(v) else v for k, v in batch.items()}

            text_emb = compute_text_embeddings(
                batch["input_ids"], batch["attention_mask"]
            )
            audio_emb = compute_audio_embeddings(
                batch["audio_values"]
            )

            logits = model(text_emb, audio_emb)
            loss = criterion(logits, batch["labels"])

            total_loss += loss.item()
            preds = logits.argmax(dim=1).cpu().numpy()

            y_pred.extend(preds)
            y_true.extend(batch["labels"].cpu().numpy())

    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average="weighted")
    return total_loss / len(data_loader), acc, f1


In [10]:
model = FusionClassifier().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss()
SAVE_DIR = "C:/Users/hp333/Desktop/Multimodel_emotion_detection/Model/checkpoints"
os.makedirs(SAVE_DIR, exist_ok=True)
checkpoint_path = os.path.join(SAVE_DIR, "fusion_checkpoint.pt")


start_epoch = 0
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=DEVICE)
    model.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    start_epoch = checkpoint["epoch"] + 1
    best_dev_f1 = checkpoint["best_dev_f1"]
    print(f"✅ Resuming training from epoch {start_epoch}")

for epoch in range(start_epoch, EPOCHS):
    model.train()
    y_true, y_pred = [], []
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Train]"):
        optimizer.zero_grad()
        text_emb = compute_text_embeddings(batch["input_ids"], batch["attention_mask"])
        audio_emb = compute_audio_embeddings(batch["audio_values"])
        logits = model(text_emb, audio_emb)
        loss = criterion(logits, batch["labels"].to(DEVICE))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        preds = torch.argmax(logits, dim=1).detach().cpu().numpy()
        y_pred.extend(preds)
        y_true.extend(batch["labels"].cpu().numpy())
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average="weighted")
    dev_loss, dev_acc, dev_f1 = evaluate(model, dev_loader, epoch)

    checkpoint_path = os.path.join(SAVE_DIR, "fusion_checkpoint.pt")
    torch.save({
        "epoch": epoch,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "best_dev_f1": dev_f1
    }, checkpoint_path)

    print("💾 Checkpoint saved (can resume training)")
    print(
        f"Epoch {epoch+1}: "
        f"Train Loss={total_loss/ len(train_loader):.4f}, Acc={acc:.4f}, F1={f1:.4f} | "
        f"Dev Loss={dev_loss:.4f}, Acc={dev_acc:.4f}, F1={dev_f1:.4f}"
    )

Epoch 1/15 [Dev]: 100%|██████████| 139/139 [01:01<00:00,  2.26it/s]


💾 Checkpoint saved (can resume training)
Epoch 1: Train Loss=1.5016, Acc=0.4702, F1=0.3110 | Dev Loss=1.5044, Acc=0.4418, F1=0.2903


Epoch 2/15 [Dev]: 100%|██████████| 139/139 [00:57<00:00,  2.42it/s]


💾 Checkpoint saved (can resume training)
Epoch 2: Train Loss=1.3527, Acc=0.5307, F1=0.4182 | Dev Loss=1.4085, Acc=0.5095, F1=0.3997


Epoch 3/15 [Dev]: 100%|██████████| 139/139 [00:58<00:00,  2.37it/s]


...

💾 Checkpoint saved (can resume training)
Epoch 13: Train Loss=1.1519, Acc=0.6149, F1=0.5663 | Dev Loss=1.2548, Acc=0.5627, F1=0.5015


Epoch 14/15 [Dev]: 100%|██████████| 139/139 [00:59<00:00,  2.35it/s]


💾 Checkpoint saved (can resume training)
Epoch 14: Train Loss=1.1476, Acc=0.6151, F1=0.5678 | Dev Loss=1.2509, Acc=0.5627, F1=0.5028


Epoch 15/15 [Dev]: 100%|██████████| 139/139 [00:57<00:00,  2.41it/s]

💾 Checkpoint saved (can resume training)
Epoch 15: Train Loss=1.1418, Acc=0.6131, F1=0.5658 | Dev Loss=1.2566, Acc=0.5627, F1=0.5026
